# iMessage

本笔记本展示如何使用iMessage聊天加载器。这个类可以帮助将iMessage对话转换为LangChain聊天消息。

在MacOS上，iMessage会将对话存储在位于`~/Library/Messages/chat.db`的sqlite数据库中（至少在macOS Ventura 13.4中是这样）。
`IMessageChatLoader`从这个数据库文件中加载数据。

1. 创建`IMessageChatLoader`并将文件路径指向您想要处理的`chat.db`数据库。
2. 调用`loader.load()`（或`loader.lazy_load()`）执行转换。可以选择使用`merge_chat_runs`来合并来自同一发送者的连续消息，和/或使用`map_ai_messages`将指定发送者的消息转换为"AIMessage"类。

## 1. 访问聊天数据库

您的终端可能被拒绝访问`~/Library/Messages`。要使用此类，您可以将数据库复制到可访问的目录（例如，Documents）并从那里加载。或者（不推荐），您可以在系统设置 > 安全和隐私 > 完全磁盘访问权限中为您的终端模拟器授予完全磁盘访问权限。

我们已经创建了一个示例数据库，您可以在[此链接的云盘文件](https://drive.google.com/file/d/1NebNKqTA2NXApCmeH6mu0unJD2tANZzo/view?usp=sharing)中使用。

In [1]:
# This uses some example data
import requests


def download_drive_file(url: str, output_path: str = "chat.db") -> None:
    file_id = url.split("/")[-2]
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

    response = requests.get(download_url)
    if response.status_code != 200:
        print("Failed to download the file.")
        return

    with open(output_path, "wb") as file:
        file.write(response.content)
        print(f"File {output_path} downloaded.")


url = (
    "https://drive.google.com/file/d/1NebNKqTA2NXApCmeH6mu0unJD2tANZzo/view?usp=sharing"
)

# Download file to chat.db
download_drive_file(url)

File chat.db downloaded.


## 2. 创建聊天加载器

为加载器提供zip目录的文件路径。您可以选择指定映射到AI消息的用户ID，以及配置是否合并消息序列。

In [2]:
from langchain_community.chat_loaders.imessage import IMessageChatLoader

In [3]:
loader = IMessageChatLoader(
    path="./chat.db",
)

## 3. 加载消息

`load()`（或`lazy_load`）方法返回"ChatSessions"列表，目前每个加载的对话仅包含一个消息列表。所有消息最初都被映射为"HumanMessage"对象。

您可以选择合并消息"序列"（来自同一发送者的连续消息）并选择一个发送者来代表"AI"。经过微调的LLM将学习生成这些AI消息。

In [9]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "Tortoise" to AI messages. Do you have a guess who these conversations are between?
chat_sessions: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="Tortoise")
)

In [13]:
# Now all of the Tortoise's messages will take the AI message class
# which maps to the 'assistant' role in OpenAI's training format
chat_sessions[0]["messages"][:3]

[AIMessage(content="Slow and steady, that's my motto.", additional_kwargs={'message_time': 1693182723, 'sender': 'Tortoise'}, example=False),
 HumanMessage(content='Speed is key!', additional_kwargs={'message_time': 1693182753, 'sender': 'Hare'}, example=False),
 AIMessage(content='A balanced approach is more reliable.', additional_kwargs={'message_time': 1693182783, 'sender': 'Tortoise'}, example=False)]

## 3. 准备微调

现在是时候将我们的聊天消息转换为OpenAI字典格式了。我们可以使用`convert_messages_for_finetuning`工具来实现这一点。

In [14]:
from langchain_community.adapters.openai import convert_messages_for_finetuning

In [15]:
training_data = convert_messages_for_finetuning(chat_sessions)
print(f"Prepared {len(training_data)} dialogues for training")

Prepared 10 dialogues for training


## 4. 微调模型

是时候微调模型了。确保您已安装`openai`
并已正确设置您的`OPENAI_API_KEY`

In [16]:
%pip install --upgrade --quiet  langchain-openai

In [18]:
import json
import time
from io import BytesIO

import openai

# We will write the jsonl file in memory
my_file = BytesIO()
for m in training_data:
    my_file.write((json.dumps({"messages": m}) + "\n").encode("utf-8"))

my_file.seek(0)
training_file = openai.files.create(file=my_file, purpose="fine-tune")

# OpenAI audits each training file for compliance reasons.
# This make take a few minutes
status = openai.files.retrieve(training_file.id).status
start_time = time.time()
while status != "processed":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    status = openai.files.retrieve(training_file.id).status
print(f"File {training_file.id} ready after {time.time() - start_time:.2f} seconds.")

File file-zHIgf4r8LltZG3RFpkGd4Sjf ready after 10.19 seconds.


文件准备好后，是时候启动训练作业了。

In [19]:
job = openai.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo",
)

在您的模型准备期间，不妨喝杯茶。这可能需要一些时间！

In [20]:
status = openai.fine_tuning.jobs.retrieve(job.id).status
start_time = time.time()
while status != "succeeded":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    job = openai.fine_tuning.jobs.retrieve(job.id)
    status = job.status

In [21]:
print(job.fine_tuned_model)

ft:gpt-3.5-turbo-0613:personal::7sKoRdlz


## 5. 在LangChain中使用

您可以直接在`ChatOpenAI`模型类中使用生成的模型ID。

In [22]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model=job.fine_tuned_model,
    temperature=1,
)

In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are speaking to hare."),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [41]:
for tok in chain.stream({"input": "What's the golden thread?"}):
    print(tok, end="", flush=True)

A symbol of interconnectedness.